# Qu'est-ce qu'un client dans le moteur d'optimisation? 
Le moteur d'optimisation est la partie (le package) du projet OptimaSol qui permet de calculer les scénatios optimisés de chauffe en prenant compte des prévisions solaire. 
La fonction de calcul du scénario optimisé prend en compte, les prévisions solaire mais aussi des paramètres client. Par exemple la taille du chauffe eau, les choix du client (ses contraintes imposées...) font aussi partie des inputs de cet algorithme d'optimisation. 

Pour utiliser le moteur d'optimisation, l'utilisateur a besoin de connaître : 

- Une classe Client qui traduit les paramètres métier du client. 

- Une classe OptimiserService qui contient la fonction du solver, le solver prend un client et génère une trajectoire.

- Les méthodes & fonctionnalités implémentées dans la classe TrajectorySystem. C'est le type de sortie du solveur et présente des services intéressants. 

Dans ce Notebook, on s'intéresse à la classe Client et à son utilisation. 
On verra : 

- La classe Client et ses briques : Ici on verra les classes imbriqués de Client, l'accès aux attributs, les méthodes proposées, les erreurs à ne pas faire. 

- La création simplifiée d'un client à partir d'un texte YAML, et à partir d'un dico Python. 




### Vue d'ensemble de la classe Client : 
Un client est une instance de la classe *Client*, il contient les données diverses relatives au client. 
Les données sont réparties dans des sous classes de cet objet. Concrètement, pour construire un client, il faut plusieurs briques (un planning, des contraintes, des fonctionnalités, des prix, un chauffe-eau et un ID). Voici un aperçu de chaque brique : 

- Un planning : C'est un élément d'une classe **Planning**, Qui est en réalité une liste de points consignes. Un point consigne est un élément d'une classe **SetPoint**. Un point consigne est un quadruplet : Jour (de la semaine, un entier de 0 à 6), Heure (un élement de format time, une température souhaitée, un volume prévu). Un planning est hebdomadaire. 

- Des contraintes : C'est un élement de la classe **Constraints**, il contient une température minimale de confort / sécurité, de possibilité de chauffe (les plages de chauffes interdites vs autorisées - C'est une liste de créneaux où la chauffe est autorisées) et un profil de consommation (c'est un tableau hebdomadaire du type **ConsumptionProfile** montrant la répartition de la consommation hors chauffe-eau).

- Fonctionnalités : Un élément de la classe **Features** qui contient l'information de Qu'est ce qu'on optimise (un élément de la classe **OptimizationMode**, si la gradation est activé ou non). 

- Prix : Un élément de la classe **Prices** contient le mode, (Basique ou bien HP / HC), ensuite les infos détaillés de chaque mode (Tarif Base ou HP /HC), tarif de revente, liste de créneaux des heures pleines (si mode HP HC). 

- Un chauffe-eau : Un élémént de la classe **WaterHeater**, contient principalement la puissance et le volume du chauffe-eau mais aussi le coefficient de pertes et la température de l'eau froide. 

- Un ID : Utile dans le cas centralisé pour la BDD, pas utilisé dans l'optimisation. 

En regroupant ces différents éléments, on crée un client. Chaque classe imbriquée contient des méthodes puissantes. 
Par exemple la classe **Waterheater** peut calculer la température dans un chauffe eau, la classe **Prices** peut dire le prix courant en prenant un instant donné... Le but de ce Notebook est d'exposer ces méthodes. 

### Comment importer : 
La classe *Client* ainsi que ses briques, s'importent à partir de optimiser_engine. 
Voici les lignes pour importer la classe Client et pour importer ses briques. 

NOTE : Si vous comptez utiliser les méthodes de création simplifiée (développée enfin), vous n'avez besoin que de la première ligne. (Vous n'importez que la classe Client). 

In [1]:
# L'import de la classe principale : 
from optimiser_engine import Client 
# Les sous classes : 
from optimiser_engine import Planning, Setpoint, Features, Constraints, ConsumptionProfile, Prices, WaterHeater, OptimizationMode, TimeSlot


# Briques de la classe Client : 
Dans cette section, nous détaillons chaque brique de la classe Client. Les briques sont aussi des classes, et parmi ces classes, il y en a ceux (comme Planning, Constraints) qui contiennent aussi des briques de classes personnalisées. 
Les classes imbriqués qui forment un client sont : 
- **Planning** 
- **Constraints**
- **Prices**
- **Features**
- **WaterHeater** 

Dans cette section, nous allons aborder chacune de ces classes. 

## 1 - Planning : 

Le planning représente en pratique un planning hebdomadaire de consignes. Une consigne comporte les informations suivantes : 
- Jour de la semaine (entier de 0 à 6). 
- Heure (un objet de time) 
- Température souhaitée (en °C) 
- Volume d'eau tiré prévu (en L). 

### 1a - La classe SetPoint : 
Le format de données des consignes est traduits par la classe **SetPoint**. 

Voici un exemple de création d'une consigne : 

In [2]:
# On commence par le jour : 
jour = 2 # Mercredi (Les jours vont de 0 à 6) 

# L'heure : 
from datetime import time 
temps = time(13,45) #13h45 (un objet de la classe time) 

#La température souhaitée : 
temperature = 70 # 70°C par exemple. 

# Volume prévu : 
volume = 30 # 30L par exemple. 

#### Instanciation de SetPoint : 

In [3]:
consigne = Setpoint(jour, temps, temperature, volume) 

NOTE : 
- On peut et on doit créer plusieurs consignes. D'ailleurs pour faire un planning complet, l'usage général est d'avoir plusieurs consignes. 
- La variable jour doit être un entier entre 0 et 6, temps une variable de type *time*, la température entre 30 et 99 et le volume un nombre, en cas de non respect de ces règles, l'initialisation lève une erreur. 

#### Accès aux attributs d'un point consigne :

In [4]:
# Pour accéder aux éléments d'une consigne : 
jour = consigne.day
heure = consigne.time
temperature = consigne.temperature
volume = consigne.drawn_volume

print(jour, heure, temperature, volume)  #Exemple pour voir. 

2 13:45:00 70 30


#### Afficher directement une consigne : 
La commande classique *print()* Celle ci renovoit un message descriptif de la consigne (en anglais) :

In [5]:
# On peut aussi printer une consigne, ça renvoit un message descriptif en anglais. 
print(consigne)

Set point : Day : Mercredi Time : 13:45:00 Temperature : 70 Drawn volume : 30


### 1b - Création d'un Planning : 

#### L'instanciation (création) :

La création du planning se fera à l'aide de la classe **Planning**. L'initialisation d'un planning prend une liste (optionnel, par défaut vide), mais ensuite on pourra ajouter des consignes, les supprimer. 

Voici un exemple de code qui crée un Planning à partir de deux consignes : 

In [6]:
#On crée deux consignes : 
consigne1 = Setpoint(1, time(20,00), 60, 45) #Mardi à 20h00, 60°C et 45L. 
consigne2 = Setpoint(4, time(7,30), 55, 40) #Vendredi à 7h30, 55°C et 40L. 

#LA liste des consignes : 
liste_consignes = [consigne1, consigne2] 
# On ajoute une consigne via la méthode add_setpoints() 
planning = Planning(liste_consignes) 

#### Ajout de consignes : 
Il est possible d'ajouter une consigne dans un planning en utilisant la méthode **add_setpoint**, voici un exemple d'utilisation :

In [7]:
#Ajout de consignes : 
planning.add_setpoint(consigne) #La consigne crée dans la dernière cellule.  

#### Afficher un planning : 
La commande print() permet de voir un message descriptif du planning :

In [8]:
#On peut printer un planning, ça renvoit un message descriptif en anglais : 
print(planning) 

[
-Mardi-20:00:00   :  60 °C - 45 L.
-Mercredi-13:45:00   :  70 °C - 30 L.
-Vendredi-07:30:00   :  55 °C - 40 L.
 ]


#### Accès aux éléments d'un planning :
On peut accéder à l'attribut *setpoints* qui est la liste des consignes, voici un exemple d'utilisation :

In [9]:
# Pour accéder à la liste des consignes : 
liste_consignes = planning.setpoints # C'est une liste de consignes et on peut accéder aux éléments internes. 
print(liste_consignes[0])
print(liste_consignes)

Set point : Day : Mardi Time : 20:00:00 Temperature : 60 Drawn volume : 45
[Set point : Day : Mardi Time : 20:00:00 Temperature : 60 Drawn volume : 45, Set point : Day : Mercredi Time : 13:45:00 Temperature : 70 Drawn volume : 30, Set point : Day : Vendredi Time : 07:30:00 Temperature : 55 Drawn volume : 40]


#### Accès aux futures consignes : 
A partir d'un planning, d'un instant (jour (entier) et heure (time)) et d'un horizon, on peut tirer toutes les consignes antérieures à cet instant qui se trouvent dans l'horizon. Pour cela on utilise la méthode **get_future_sepoints**, voici un exemple d'utilisation : 

In [10]:
#Pour récupérer les prochaines consignes à partir d"une date, et jusqu'à un certain horizon :
jour = 1 #On va capturer la consigne de mardi. 
heure = time(7,0) # 18h car on va capturer la consigne de 20h 
horizon = 24 # 24h. 
liste_futures_consignes = planning.get_future_setpoints(jour, heure, horizon) 
print(liste_futures_consignes)

[Set point : Day : Mardi Time : 20:00:00 Temperature : 60 Drawn volume : 45]


#### Suppression d'une consigne : 
En disposant du jour (entier de 0 à 6) et l'heure (time), on supprimer la consigne de cette date (si elle existe). 
Pour cela on utilise la méthode **remove_setpoints** qui supprime la consigne et renvoit *True* si cette consigne existe, ne fait rien et renvoit *False* s'il n'y a aucune consigne enregistrée dans cette date. Voici un exemple d'utilisation : 

In [11]:
#Pour la suppression d'une consigne, il suffit d'indiquer sa date / heure : 
jour = 1 #Mardi 
heure = time(20,0)
print(planning)
planning.remove_setpoint(jour, heure)
#Maintenant le planning ne contient pas la consigne de Mardi - 20h 
print(planning)

[
-Mardi-20:00:00   :  60 °C - 45 L.
-Mercredi-13:45:00   :  70 °C - 30 L.
-Vendredi-07:30:00   :  55 °C - 40 L.
 ]
[
-Mercredi-13:45:00   :  70 °C - 30 L.
-Vendredi-07:30:00   :  55 °C - 40 L.
 ]


#### Suppression de toutes les consignes : 
On peut supprimer toutes les consignes en utilisant la méthode **clear()**. Voici un exemple d'utilisation : 

In [12]:
#Pour supprimer toutes les consignes d'un planning :
print(planning) 
planning.clear() 
print(planning) 

[
-Mercredi-13:45:00   :  70 °C - 30 L.
-Vendredi-07:30:00   :  55 °C - 40 L.
 ]
[
 ]


## 2 - Les contraintes : 

Les contraintes sont décrites par une classe **Constraints**. Un objet de cette classe contient les informations suivantes : 
- Un tableau N*7 de la classe ConsumptionProfile (représente la consommation de la maison globale (à part le chauffe eau)).
- Une liste de créneaux interdites. (le chauffe eau va être en mode OFF durant ces plages).
- Une température minimale (l'eau ne doit pas descendre en dessous de celle ci). 

### 2a - Le profil de la consommation : 
Le profil est un objet de la classe **ConsumptionProfile**. Cet objet est essentiel, il représente la consommation globale de la maison (chauffe-eau non inclus). Ce profil est traduit par un tableau (Nx7) (7 pour 7 jours dans la semaine - C'est un profil périodique sur une semaine). $N$ représente le nombre de points par jour. Par défaut ce nombre vaut 24, mais il peut être modifié. 

Pour l'instanciation, il y a deux choix : 
- On fournit un tableau (np.ndarray) de (N*7) 
- On fournit un bruit de fond (une puissance constante (par défaut 300W)) et le profil sera rempli de cette puissance partout. 

La première méthode est plus précise mais plus pénible. La deuxième est moins réaliste et n'exploite pas tout le potentiel du package mais elle est plus rapide. 

#### Modification du nombre de points : 
Le nombre de point par jour (où on interpole la consommation) peut être modifié en accédant à *points_per_day* qui est une attribut de la classe *ConsumptionProfile*. 
NOTE : Il s'agit d'un attribut de la classe et non pas des instances car c'est un paramètre qui ne concerne pas le client. 
Voici un exemple de modification : 

In [13]:
# Exemple, modification du nombre de Points : 
ConsumptionProfile.points_per_day = 30 # On a mis 30 au lieu de 24. 
# ATTENTION : points_per_day est une attribut de classe pas d'instance. 
# On remet la valeur initiale 24 : 
ConsumptionProfile.points_per_day = 24 

#### Création d'un profil de consommation :
Un profil de consommation est un tableau de N*7 points. 
La classe **ConsumptionProfile** peut être initialisée par un tableau de cette dimension de type *np.ndarray*. 
On importe d'abord *numpy* :

In [14]:
import numpy as np #Essentiel car le tebleau qu'on utilise est un numpy.ndarray. 

On génère un tableau exemple avec des valeurs Random, sachant que n'importe qu'elle tableau réaliste qui vérifie les bonnes dimensions est valable. 

In [15]:

# On va remplir le profil par un tableau avec valeurs aléatoires (c'est juste un exemple) 
rng = np.random.default_rng()
tab_consommation = rng.integers(low=0, high=401, size=(7, ConsumptionProfile.points_per_day))  
print(tab_consommation)

[[ 32 177   2 351  15 150 305  50 355 197 132  31 393 212 363  47 307 356
   64 354 141 142 290 175]
 [  2 184 222 349 300 160 313 400 388 167  35  19 314 382 223 117 167  67
  106  54  38 105 374 141]
 [ 65   8 253 156 141 193 322 356 224 259 350 362  85 281 265 105 296 116
  291 336 332 228  95 195]
 [111  60 352 200  79 281 364 282  76 254 257 392 315 141  88 307   4 172
  213 383 323 318 132 132]
 [261 199 378  58 384 292 242 331 333  36 325 193  78 110 238 257 221 142
  195 108  37 132  61 255]
 [ 33  81 193 347 369 189 256 235 218 289 339 352 258 371 296 175 149 227
  132 364  10 161 327 297]
 [216 334 320  28 289  43 164 123 380 179 217 270 342 251 115 270 227  25
  170 140 181 326 230 125]]


Et voici un exemple d'instanciation de ConsumptionProfile qui repose sur l'intialisation par tableau détaillé. 

NOTE : Une valeur irréaliste (consommation négative, non respect des types ou bien un non respect des dimensions) entraîne des erreurs levées. 

L'affiche du profil pourrait se faire avec print(). 

In [16]:
# On instancie avec tab_consommation.
profil = ConsumptionProfile(tab_consommation)
print(profil)

The consumption profile : 
- Background noise : 300.0
- Table of consumption : 
 [[ 32 177   2 351  15 150 305  50 355 197 132  31 393 212 363  47 307 356
   64 354 141 142 290 175]
 [  2 184 222 349 300 160 313 400 388 167  35  19 314 382 223 117 167  67
  106  54  38 105 374 141]
 [ 65   8 253 156 141 193 322 356 224 259 350 362  85 281 265 105 296 116
  291 336 332 228  95 195]
 [111  60 352 200  79 281 364 282  76 254 257 392 315 141  88 307   4 172
  213 383 323 318 132 132]
 [261 199 378  58 384 292 242 331 333  36 325 193  78 110 238 257 221 142
  195 108  37 132  61 255]
 [ 33  81 193 347 369 189 256 235 218 289 339 352 258 371 296 175 149 227
  132 364  10 161 327 297]
 [216 334 320  28 289  43 164 123 380 179 217 270 342 251 115 270 227  25
  170 140 181 326 230 125]]


Voici un deuxième exemple qui crée un profil sans tableau détaillé mais juste une valeur constante partout. 

In [17]:
# Si on dispose pas d'un tableau, on peut instancier juste avec une valeur constante via le bruit de fond, qu'on peut personnaliser : 
profil2 = ConsumptionProfile(background_noise=500)
print(profil2)

The consumption profile : 
- Background noise : 500
- Table of consumption : 
 [[500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
  500. 500. 500. 500. 500. 500. 500. 500. 500. 500.]
 [500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
  500. 500. 500. 500. 500. 500. 500. 500. 500. 500.]
 [500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
  500. 500. 500. 500. 500. 500. 500. 500. 500. 500.]
 [500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
  500. 500. 500. 500. 500. 500. 500. 500. 500. 500.]
 [500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
  500. 500. 500. 500. 500. 500. 500. 500. 500. 500.]
 [500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
  500. 500. 500. 500. 500. 500. 500. 500. 500. 500.]
 [500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500. 500.
  500. 500. 500. 500. 500. 500. 500. 500. 500. 500.]]


#### Affichage des puissances antérieures 

A partir d'une date, on peut générer le vecteur des "puissances suivantes", pour cela il faut fournir : 
- Une date (format datetime) 
- un nombre de Points. 
- Le pas de temps 
On utilise la méthode **get_vector** qui interpole linéairement le tableau de profil et génère le vecteur. 


Voici un exemple de code qui génère un vecteur consitutée de N puissances de consommation après une date et avec un pas de temps défini. 

NOTE : L'horizon du vecteur peut être calculé en multipliant N par le pas. 

In [18]:
from datetime import datetime
date = datetime(2026,1,1,13,45) 
N = 48 
pas = 30 
puissances = profil.get_vector(date, N, pas)
print(puissances)

[101.25 142.75 252.25 231.25  79.75  46.   130.   182.25 202.75 255.5
 340.5  368.   338.   321.75 319.25 271.5  178.5  132.   132.   164.25
 228.75 245.5  214.5  243.75 333.25 298.   138.   139.5  302.5  361.
 315.   279.5  254.5  264.25 308.75 331.5  332.5  258.75 110.25 108.25
 252.75 292.   226.   164.25 106.75  86.   102.   142.  ]


### 2b - Plages de chauffe eau interdites : 
Ils représentent les créneaux où le chauffe eau n'est pas autorisé à se mettre en mode ON. 
Ils sont représentés par une liste de créneaux. Un créneau est un objet de la classe **TimeSlot**
Pour instancier les contraintes, il faut donner une liste de créneaux. 
Ces créneaux doivent respecter des règles : 
- Pas de chevauchement entre deux créneaux. 
- Les créneaux ne doivent pas couvrir tout les 24h. 

NOTE : Les plages interdites sont périodiques sur 24h, il est pas possible de définir une plage de chauffe différente entre deux journées. 


Voici un exemple de code qui crée deux créneaux, ensuite une liste composées de ces créneaux : 

In [19]:
#On instancie deux créneaux (Exemple) : 
creneau1 = TimeSlot(time(8,30), time(11,20)) # De 08h30 à 11h20. 
creneau2 = TimeSlot(time(20,0), time(23,45)) # De 20h00 à 23h45. 

liste_creneaux = [creneau1, creneau2] 

### 2c - La température minimale : 
Il s'agit de la 3e et la dernière composante de l'objet Constraints. C'est une température minimale que le client définit. L'eau ne descendra pas au dessous de cette température dans le chauffe eau. 

NOTE : 
- Il n'est pas possible de définir une température minimale au dessous de 0°C ou bien au dessus de 95°C. 

Il s'agit simplement d'un flottant qu'on peut définir ainsi : 

In [20]:
temperature_minimale = 40

### 2d - Assemblage dans l'objet contraintes : 
Après avoir défini le profil (*ConsumptionProfile*), la liste des créneaux de plages interdites, la température minimale, on peut créer l'objet contraintes. 
Voici un exemple d'instanciation qui repose sur les briques crées précédemment : 

In [21]:
contraintes = Constraints(profil, liste_creneaux, temperature_minimale)

### 2e - Manipulation des contraintes : 
Ci dessous des exemples de codes pour accéder aux plages interdites, ajouter une plage interdite ... 

#### Accès à la liste de plages interdites : 
L'accès se fait via l'attribut *forbidden_slots*, voici un exemple d'appel :

In [22]:
# L'accès aux plages interdites se fait via l'attribut forbidden_slots : 
plages = contraintes.forbidden_slots
print(plages)

[[08:30 - 11:20], [20:00 - 23:45]]


#### Accès au profil de consommation
L'accès au profil de consommation se fait via l'atribut *consumption_profile*, voici un exemple qui traduit cela :

In [23]:
# L'accès au profil de consommation se fait via l'attribut consumption_profile : 
profil_consommation = contraintes.consumption_profile
print(profil_consommation)

The consumption profile : 
- Background noise : 300.0
- Table of consumption : 
 [[ 32 177   2 351  15 150 305  50 355 197 132  31 393 212 363  47 307 356
   64 354 141 142 290 175]
 [  2 184 222 349 300 160 313 400 388 167  35  19 314 382 223 117 167  67
  106  54  38 105 374 141]
 [ 65   8 253 156 141 193 322 356 224 259 350 362  85 281 265 105 296 116
  291 336 332 228  95 195]
 [111  60 352 200  79 281 364 282  76 254 257 392 315 141  88 307   4 172
  213 383 323 318 132 132]
 [261 199 378  58 384 292 242 331 333  36 325 193  78 110 238 257 221 142
  195 108  37 132  61 255]
 [ 33  81 193 347 369 189 256 235 218 289 339 352 258 371 296 175 149 227
  132 364  10 161 327 297]
 [216 334 320  28 289  43 164 123 380 179 217 270 342 251 115 270 227  25
  170 140 181 326 230 125]]


#### Accès à la température minimale : 
L'accès à la température minimale dans un objet contraintes se fait via l'attribut *minimim_temperature*, voici un exemple :

In [24]:
# L'accès à la température minimale se fait via l'attribut minimum_temperature : 
temp_min = contraintes.minimum_temperature
print(temp_min) 

40


#### Ajout d'une plage interdite : 
On peut ajouter une plage interdite de chauffe via la méthode *add_forbidden_slot()*, celle-ci prend en argument deux objets *time* (début, fin).  
NOTE : Il faut toujours respecter la règle de non chevauchement avec les plages existantes et l'ensemble des plages ne doivent pas couvrir toute la journée. 
Voici un exemple de code de l'ajout d'une plage interdite :

In [25]:
# Ajouter une plage interdite : 

# On appelle la méthodes add_forbidden_slot, on y met les instants (time) de début et de fin. 
# Il faut être sûr de respecter les conditions de non chevauchement. 
contraintes.add_forbidden_slot(time(13,0), time(15,30))
print(contraintes.forbidden_slots) # Le créneau est bien ajouté. 

[[08:30 - 11:20], [13:00 - 15:30], [20:00 - 23:45]]


#### Accès à la possibilité de chauffe :
Pour savoir si on peut chauffer durant un instant time(), on peut appeler la méthode *is_allowed* qui prend en argument un objet *time*. Voici un exemple d'utilisation de cette méthode. 

In [26]:
# Pour savoir si on est autorisé à chauffer ou non : 
# On appelle la méthode is_allowed qui renvoie un booléen: 

print(contraintes.is_allowed(time(13,45))) #False car 13h45 est entre 13h et 15h30. 
print(contraintes.is_allowed(time(12,00))) # True car 12h00 n'est pas dans un créneau interdit. 

False
True


## 3 - Les Features (Les fonctionnalités) : 
Cette classe stocke les informations : 
- Le mode gradation est activé ou pas. 
- Le mode de l'optimisation (on optimise quoi? Le coût ou bien l'optimisation.) 

### Création d'un objet Features :

Pour instancier des fonctionnalités via la classe **Features**, il faut simplement choisir le mode d'optimisation via la classe OptimizationMode (c'est une classe Enum, elle a deux états COST ou AUTOCONS). 
Voici un exemple qui instancie deux features différentes : 

In [27]:
gradation1 = True 
gradation2 = False
mode1 = OptimizationMode.COST 
mode2 = OptimizationMode.AUTOCONS
feature1 = Features(gradation1, mode1)        #Exemple : Gradation activée et on optimise le coût. 
feature2 = Features(gradation2, mode2)        #Exemple : Gradation désactivée et on optimise l'autoconsommation.

### Accès aux deux attributs et affichage :

L'accès au mode et à la gradation se fait via les attributs *gradation* et *mode*, on peut aussi faire un print pour visualiser le feature. ça renvoie un message descriptif en anglais. 

In [28]:
print(feature1.gradation) 
print(feature2.mode) 
print(feature1) 
print(feature2)

True
OptimizationMode.AUTOCONS
- Gradation : ON
- Quantity optimized : Cost (€)
- Gradation : OFF
- Quantity optimized : Autoconsumption


## 4 - Les prix : 
Cette classe (brique de client) contient les informations : 
- Mode de tarification : Basique ou bien tarif HP / HC. 
- Dans chaque cas les tarifs. 
- Dans le cas de HP / HC, les créneaux de HP vs ceux de HC. 
- Le tarif de revente. 

Elle contient la méthode *get_current_purchase_price* qui peut donner le prix d'achat actuel en prenant une heure (*time*). 

### Création d'un objet Prices :
Ci dessous, deux exemples de deux instances, une avec mode Basic et une mode HP HC. 

NOTE : 
- Une fois on choisit le mode, il devient impossible d'accéder ou de mettre à jour un attribut qui concerne l'autre mode. Par exemple, si *prix1* est une instance avec le mode BASE, vouloir mettre un prix HP / HC, génèrera une erreur **ModeIncompatibleError**. 
- Le mode dans la classe Prices doit être une chaîne de caractères (soit "BASE" soit "HPHC"), une valeur erronée engendre une ValueError.

L'initialisation requiert le mode (par défaut on met Basique), ensuite la modification des tarifs s'effectue en accédant aux attributs. 

Exemple 1 : Tarif Basic : 

In [29]:
#Initialisation : 
prix_base = Prices("BASE") 

#On met le tarif de Basic : 
prix_base.base = 0.20 #0.2 €. 

#On met le tarif de revente : 
prix_base.resale_price = 0.10 #0.10 €

print(prix_base.get_current_purchase_price(time(23,00))) #Accès au prix à 23h -> 0.2 car c'est basic. 

print(prix_base)     #Visualiser les prix. 

0.2
-Mode : Basic.
-Purchase price (prix d'achat) : 0.2
The price is constant because the mode is Basic.
-Resale price (Prix de revente) : 0.1


Exemple 2 : Cas HP HC : 

In [30]:
prix_hphc = Prices("HPHC") 

#On met le tarif HP HC : 
prix_hphc.hp = 0.22
prix_hphc.hc = 0.18 

#Tarif de revente : 
prix_hphc.resale_price = 0.12 

#Intervalles de HP HC, cela doit être une liste de créneaux :
prix_hphc.hp_slots = [TimeSlot(time(8,0), time(12,0)), TimeSlot(time(16,0), time(22,0))] #ici les créneaux HP varient de 8h -> 12h // 16h -> 22h

print(prix_hphc) 

-Mode : HP-HC (Heures pleines / Heures creuses)
-Purchase price HP : 0.22 
-Purchase price HC : 0.18
-The slots of HP : 
 [[08:00 - 12:00], [16:00 - 22:00]]
-Resale price (Prix de revente) : 0.12


### Accès au tarif actuel : 
L'accès au tarif pour un instan *time* se fait via la méthode *get_purchase_price* qui prend en argument un objet de type *time*, voici un exemple d'utilisation : 

In [31]:
# Accéder au tarif actuel : #C'est là que la méthode get_current_purchase_price devient intéressante; 
print(prix_hphc.get_current_purchase_price(time(9,0))) 
print(prix_hphc.get_current_purchase_price(time(13,0)))

0.22
0.18


## 5 - Les paramètres du chauffe-eau : 
Cette dernière classe (brique de client) contient les informations relatives au chauffe eau.
Un chauffe eau peut être initialisé par : 
- Puissance nominale. 
- Volume. 

Après initialisation, on peut modifier : 

- Coefficient d'isolation (ou bien de pertes (en °C/min)) (Par défaut mise à 0)
- Température d'eau froide (par défaut 15°C si pas de modification.) 

Cette classe propose une méthode *calculate_temperature* qui calcule la température après une durée pour une température initiale et un volume d'eau tiré. 

### Création d'un objet WaterHeater :
Voici un exemple de code qui crée un chauffe eau, ensuite personalise les paramètres. 

In [32]:
chauffe_eau = WaterHeater(volume = 300, power = 1500) #Puissance 1500 W et volume = 300L. 
#On modifie le coefficient d'isolation : 
chauffe_eau.insulation_coefficient = 0.02 #Perte de 0.02 °C / min. 
#On modifie l'eau froide : 
chauffe_eau.cold_water_temperature = 12 # 12°C. 

print(chauffe_eau)

Water Heater : 
-Volume (Volume): 300
-Nominal Power (Puissance nominale): 1500
-Insulation coefficient (coefficient de pertes) : 0.02 °C/min
-Cold Water considered (Eau froide): 12 °C


### Calcul d'une température :
Après un intervalle où connait la température initiale, la proportion de la puissance injectée (supposée constante durant l'intervalle), la durée de l'intervalle et le volume d'eau tiré (remplacé par l'eau froide), on calcule la nouvelle température dans le chauffe eau via la méthode : *calculate_temperature*, voici un exemple d'utilisation : 

In [33]:
# La méthode de calcul de la température : 
nouvelle_temperature = chauffe_eau.calculate_temperature(temp_init=50, #Température initiale. 
                                                         power_ratio=0.6, # la proportion de puissance injectée; 
                                                         time_delta_minutes=60, #60minutes. 
                                                         drawn_volume=80 #On a tiré 80 litres. 
                                                         ) 
print(nouvelle_temperature)



41.24731182795699


# Assemblage de client : 
Maintenant on a présenté toutes les briques, un client est un ensemble constitué de chacun de ces briques et un ID. 

## Création d'un client : 
Cela s'effectue simplement via initialisation par les briques crées. 
Ci dessous un exemple qui crée un client par cette méthode en utilisant des briques crées avant comme exemple : 

In [34]:
client = Client(planning = planning, constraints=contraintes, features=feature1, prices=prix_hphc, water_heater=chauffe_eau, client_id=10) 

print(client) #Affiche le client_id

client : 10


## Accès aux attributs : 
Les 5 briques du client deviennent des attributs du client, elles sont accessibles via la méthode standard standard en Python. Voici un exemple de code qui accède à ces briques et les affichent :

In [35]:
# Accès aux éléments internes : 
print(client.constraints) 
print(client.features) 
print(client.prices) 
print(client.planning) 
print(client.water_heater)

<Constraints: Restriction on [[08:30 - 11:20], [13:00 - 15:30], [20:00 - 23:45]]>
<Minimal Temperature : 40
 -Please type print(self.consumption_profile) to access to the details of profile consumption
- Gradation : ON
- Quantity optimized : Cost (€)
-Mode : HP-HC (Heures pleines / Heures creuses)
-Purchase price HP : 0.22 
-Purchase price HC : 0.18
-The slots of HP : 
 [[08:00 - 12:00], [16:00 - 22:00]]
-Resale price (Prix de revente) : 0.12
[
 ]
Water Heater : 
-Volume (Volume): 300
-Nominal Power (Puissance nominale): 1500
-Insulation coefficient (coefficient de pertes) : 0.02 °C/min
-Cold Water considered (Eau froide): 12 °C


# Méthodes simplifiées de création d'un client : 
Vu que la création d'un client par des factories est pénible (il faut créer les 5 briques ...). Il y a des méthodes qui facilitent cette tâche : 

- Une méthode qui crée un client à partir d'un YAML type (ou bien d'un fichier YAML). 
- Une méthode qui crée un client à partir d'un Dictionnaire exemple. 

Évidemment, une fois un client créé, les méthodes / accès aux données précédentes restent valides. Cela s'applique également pour les méthodes relatives aux classes internes (imbriquées). 

## 1 - A partir d'un YAML : 
On peut décrire un client tout entier dans un texte YAML, ensuite appeler une méthode (@classmethod) *from_yaml* qui génère un client à partir de cette description. 
Cependant, le texte YAML doit respecter un format standard. 
 

### Format standard de texte YAML à respecter :
Voici le format standard YAML à respecter :

In [36]:
config_yaml = """
# Configuration d'un Client
client_id: 12345

water_heater:
  volume: 200.0            # En Litres
  power: 2500.0            # En Watts
  insulation_coeff: 0   # Optionnel (défaut 0). Perte en °C/min
  temp_cold_water: 10.0    # Optionnel (défaut 10). En °C

prices:
  mode: "HPHC"             # Choix : "BASE" ou "HPHC"
  hp_price: 0.25           # Prix du kWh en Heures Pleines
  hc_price: 0.15           # Prix du kWh en Heures Creuses
  #base_price: 0.20         # Utilisé seulement si mode = "BASE"
  resell_price: 0.10       # Prix de revente
  # Liste des créneaux HP (Format "HH:MM", obligatoire si mode HPHC)
  hp_slots:
    - start: "06:00"
      end: "22:00"

features:
  gradation: true          # true si le chauffe-eau supporte la modulation de puissance
  mode: "cost"             # Choix : "cost" (économique) ou "AutoCons" (autoconsommation)

constraints:
  min_temp: 40.0           # Température minimale de sécurité
  # Périodes où la chauffe est INTERDITE
  forbidden_slots:
    - start: "12:00"
      end: "14:00"
  
  # Profil de consommation (null = profil par défaut / bruit de fond)
  # Pour un profil personnalisé, fournir une liste de 7 listes de 24 valeurs
  consumption_profile: null
  background_noise : 400 #optionnel, pour une maison à puissance constante. 

planning:
  # Liste des besoins en eau chaude
  # day : 0 = Lundi, ... 6 = Dimanche
  - day: 0
    time: "18:15"          # Mettre entre guillemets pour éviter la confusion avec des minutes
    target_temp: 55.0
    volume: 10.0

  - day: 1
    time: "18:15"
    target_temp: 55.0
    volume: 10.0

  - day: 2
    time: "08:00"
    target_temp: 55.0
    volume: 10.0

  - day: 2
    time: "18:30"
    target_temp: 55.0
    volume: 10.0

  - day: 3
    time: "18:15"
    target_temp: 55.0
    volume: 10.0

  - day: 4
    time: "09:00"
    target_temp: 55.0
    volume: 10.0

  - day: 4
    time: "18:15"
    target_temp: 55.0
    volume: 10.0

  - day: 5
    time: "10:30"
    target_temp: 55.0
    volume: 10.0

  - day: 5
    time: "17:00"
    target_temp: 55.0
    volume: 10.0

  - day: 6
    time: "09:00"
    target_temp: 55.0
    volume: 10.0

  - day: 6
    time: "18:15"
    target_temp: 55.0
    volume: 10.0 """


### Appel de la méthode de création :
La méthode à appeler est une *classmethod*, pour créer un client il faut appeler : from_yaml(). Voici l'exemple : 

In [37]:
client_cree = Client.from_yaml(config_yaml) 

On peut accéder aux mêmes attributs de Client, les mêmes méthodes internes explicités dans la partie précédente. 

Exemple :

In [38]:
# Exemple d'accès aux attributs internes. 
print(client_cree) 
print(client_cree.constraints) 
print(client_cree.features)

client : 12345
<Constraints: Restriction on [[12:00 - 14:00]]>
<Minimal Temperature : 40.0
 -Please type print(self.consumption_profile) to access to the details of profile consumption
- Gradation : ON
- Quantity optimized : Cost (€)


### A partir d'un fichier YAML :
A partir d'un fichier .yaml, on appelle simplement la fonction from_yaml_file(path). path doit être un objet de type Path défini dans Pathlib. 

## 2 - A partir d'un dictionnaire : 
Cela est moins pratique, mais un dictionnaire type, peut parfaitement devenir un client via la classmethod (from_dict), voici un dict typique. Cela suit la même logique que le YAML. En réalité, la fonction *from_yaml* traduit d'abord le YAML en DICO avant d'appliquer *from_dict*. Les YAML restent plus lisibles. 


### Format standard des dictionnaires acceptés :
Pour que la méthode *from_dict* réussisse, il faut respecter ce format standard donné dans cet exemple :

In [39]:
# Dictionnaire de configuration complète d'un client


client_config_dict = {
    # Identifiant unique du client
    "client_id": 12345,

    # Configuration physique du chauffe-eau
    "water_heater": {
        "volume": 200.0,            # Volume en Litres
        "power": 2500.0,            # Puissance en Watts
        "insulation_coeff": 0,      # (Optionnel) Coefficient d'isolation (Perte en °C/min)
        "temp_cold_water": 10.0     # (Optionnel) Température de l'eau froide réseau en °C
    },

    # Configuration tarifaire
    "prices": {
        "mode": "HPHC",             # Mode tarifaire : "BASE" ou "HPHC"
        "hp_price": 0.25,           # Prix du kWh en Heures Pleines
        "hc_price": 0.15,           # Prix du kWh en Heures Creuses
        # "base_price": 0.20,       # (Optionnel) Utilisé uniquement si mode = "BASE"
        "resell_price": 0.10,       # Prix de revente du surplus solaire
        
        # Liste des créneaux d'Heures Pleines (Obligatoire si mode = "HPHC")
        "hp_slots": [
            {"start": "06:00", "end": "22:00"}
        ]
    },

    # Fonctionnalités et mode d'optimisation
    "features": {
        "gradation": True,          # True si le chauffe-eau supporte la modulation, False sinon
        "mode": "cost"              # Objectif : "cost" (économique) ou "AutoCons" (autoconsommation)
    },

    # Contraintes opérationnelles
    "constraints": {
        "min_temp": 40.0,           # Température minimale de sécurité/confort à maintenir
        
        # Périodes où la chauffe est strictement INTERDITE
        "forbidden_slots": [
            {"start": "12:00", "end": "14:00"}
        ],
        
        # Profil de consommation d'eau chaude (None = profil par défaut)
        "consumption_profile": None,
        
        # Consommation de fond de la maison en Watts (optionnel)
        "background_noise": 400
    },

    # Planning hebdomadaire des besoins en eau chaude
    # day : 0 = Lundi, 1 = Mardi ... 6 = Dimanche
    "planning": [
        {"day": 0, "time": "18:15", "target_temp": 55.0, "volume": 10.0}, # Lundi
        {"day": 1, "time": "18:15", "target_temp": 55.0, "volume": 10.0}, # Mardi
        
        # Mercredi (2 puisages)
        {"day": 2, "time": "08:00", "target_temp": 55.0, "volume": 10.0},
        {"day": 2, "time": "18:30", "target_temp": 55.0, "volume": 10.0},
        
        {"day": 3, "time": "18:15", "target_temp": 55.0, "volume": 10.0}, # Jeudi
        
        # Vendredi (2 puisages)
        {"day": 4, "time": "09:00", "target_temp": 55.0, "volume": 10.0},
        {"day": 4, "time": "18:15", "target_temp": 55.0, "volume": 10.0},
        
        # Samedi (2 puisages)
        {"day": 5, "time": "10:30", "target_temp": 55.0, "volume": 10.0},
        {"day": 5, "time": "17:00", "target_temp": 55.0, "volume": 10.0},
        
        # Dimanche (2 puisages)
        {"day": 6, "time": "09:00", "target_temp": 55.0, "volume": 10.0},
        {"day": 6, "time": "18:15", "target_temp": 55.0, "volume": 10.0}
    ]
}

### Appel de la méthode de création :
Voici un exemple de création d'un client à partir de la méthode *from_dict*. 

In [40]:
# Appel de from_dict : 

client_cree_dict = Client.from_dict(client_config_dict) 

### Accès aux attributs / méthodes :
Cela reste évidemment le même pour tout client, voici un exemple :


In [41]:
print(client_cree_dict)
print(client_cree_dict.features) # Exemple d'accès. 

client : 12345
- Gradation : ON
- Quantity optimized : Cost (€)
